In [1]:
import pandas as pd
import numpy as np

In [2]:
################### generate emission parameters ##############################
# function to load training set to generate emission table
def load_train(train_path):
    f = open(train_path, encoding="utf8")
    lines = []
    for line in f:
        if line != '\n':
            line = line.strip('\n').split(' ')
            lines.append(line)
    df= pd.DataFrame(lines, columns = ['word', 'state'])
    return df

# function to load dev.in to generate emission table
def load_test(test_path):
    f = open(test_path, encoding='utf8')
    lines = []
    for line in f:
        if line != '\n':       
            line = line.strip('\n')
            lines.append(line)
    df= pd.DataFrame(lines, columns = ['word'])
    return df


def gen_emission_param_table_UNK(training_data_path, test_data_path):
    training_data = load_train(training_data_path)
    unique_word_list_training = training_data.word.unique()
    unique_state_list_training = training_data.state.unique()
    
    
    
    test_data = load_test(test_data_path)
    unique_word_list_test = test_data.word.unique()
    
    unk_list = np.setdiff1d(unique_word_list_test, unique_word_list_training) # return the list of words in test data but not in training data
    #non_unk_list_test = np.setdiff1d(unique_word_list_test, unk_list) # return the list of non UNK words in test data
    
    data = {word:(np.zeros(len(unique_state_list_training))) for word in unique_word_list_training}
    data["UNK"] = np.zeros(len(unique_state_list_training))    # add a UNK column to the table
    
    emission_count_table = pd.DataFrame(data, index = unique_state_list_training)           # transform the dictionary into colums with index as name of each state(y),  
                                                                                   # columns as each word of x, all entries are 0
    
    y_count_dic = {state:0 for state in unique_state_list_training}                         # y_count_dic stores Count(y) in a dictionary
    emission_param_table = pd.DataFrame(data, index = unique_state_list_training)           # emission_count_table stores all Count(y -> x) in a dataframe
                                                                                   # emission_param_table stores all the emission parameters in a dataframe
    
    print("updating emission_count_table and y_count_dic")
    for index, row in training_data.iterrows():
        word = row['word']
        state = row['state']
        #print(index, word, state)
        #print(index)
        y_count_dic[state]+=1
        if word not in unk_list:
            emission_count_table[word][state] += 1
        
    
    print("updating emission_param_table")
    k = 0.5
    for index, row in training_data.iterrows():
        word = row['word']
        state = row['state']
        #print(index)
        if word not in unk_list:
            emission_param_table[word][state] = emission_count_table[word][state] / (y_count_dic[state] + k)    
    for state in unique_state_list_training:
        emission_param_table['UNK'][state] = k/(y_count_dic[state] + k)    # compute the UNK value for each state y

    
    print("unl_list is: ",unk_list)
    print("y_count_dic is: ", y_count_dic)
    return emission_param_table, unk_list



In [3]:
def get_emission_parameter_UNK(emission_param_table, unk_list, x, y):

    if x in unk_list:
        result = emission_param_table['UNK'][y]
        #print(f"{x} is tagged as UNK and the this e('UNK'|{y}) is {result}" )
        return result
    elif x not in emission_param_table.columns:
        #print(f"word {x} is not found in the test set")
        return 
    result = emission_param_table[x][y]
    return result

In [4]:
################### generate transition parameters ##############################

# load_train here is different from part 2
# read the train data line by line, replace each '\n' with STOP and START
# so that it is clear when each sentence end and when each sentence started
def load_train_transition(train_path):
    f = open(train_path, encoding="utf8")
    lines = []
    # add a START to before first sentence
    lines.append(['START'])
    for line in f:
        if line == '\n':
            lines.append(['STOP'])
            lines.append(['START'])
        
        else:
            line = line.strip('\n').split(' ')
            del line[0]  # keep the states only
            lines.append(line)
        
    df= pd.DataFrame(lines, columns = ['state'])
    return df

def create_transition_count_table(df_train):
    unique_state_list = df_train.state.unique()
    
    data = {state:(np.zeros(len(unique_state_list))) for state in unique_state_list}
    transition_count_table = pd.DataFrame(data, index = unique_state_list)   # there will one extra STOP row and a START column
    transition_count_table = transition_count_table.drop('STOP')  #drop the extra STOP column
    transition_count_table = transition_count_table.drop(columns=['START']) #drop the extra START column
    return transition_count_table

def create_y_count_dic(df_train):
    unique_state_list = df_train.state.unique()
    y_count_dic = {state:0 for state in unique_state_list}
    y_count_dic.pop('STOP', None)  # remove the extra STOP state since we are inclusing count(STOP) when computing the transition paraameter 
    return y_count_dic

def gen_transition_param_table(train_path):
    input_data = load_train_transition(train_path)
    #transition_count_table,  y_count_dic = create_TransitionCountTable_and_YCountDic(input_data)
    transition_count_table = create_transition_count_table(input_data)
    transition_param_table = transition_count_table.copy(deep=True) # create a empty transition_param_table. Rows and columns of transition_param_table same as transition_count_table
    y_count_dic = create_y_count_dic(input_data)
    print('Generating transition parameter table')
    for i in range(len(input_data) -1):          # len(input_data) -1 coz we iterating from y_i = 0 to y_i = n-1
        y_i = input_data['state'][i]
        y_i_p1 = input_data['state'][i+1]        # y_i_p1 stands for yi_+1 (y_i plus 1)
        if y_i != 'STOP':                        # we do not count the transition from STOP to some other state
            transition_count_table[y_i_p1][y_i] += 1
            y_count_dic[y_i] += 1
            
    cols_list = transition_count_table.columns.values.tolist()      
    index_list = transition_count_table.index.values.tolist()
    for index in index_list:
        for col in cols_list:
            transition_param_table[col][index] = transition_count_table[col][index]/y_count_dic[index]  # a(y_i, y_i+1) = Count(y_i, y_i+1) / Count(y_i)
    return transition_param_table



In [5]:
# load dev.in, identify the sentences. store each sentence as a list
# return a nested list sentence_list stroing all sentences (list)
def get_sentence_list(test_path):
    f = open(test_path, encoding="utf8")
    sentence_list = [] # a list of all sentences
    sentence = []   # a list storing 1 sentence
    for word in f:
        if word != '\n':
            word = word.strip('\n')
            sentence.append(word)
        
        else:
            sentence_list.append(sentence)
            sentence = []
    return sentence_list

sentence_list = get_sentence_list('SG/dev.in')
# for i in range(5):
#     print('##########################################')
#     display(sentence_list[i])

def get_state_list(train_path):
    f = open(train_path, encoding="utf8")
    state_list = []
    # add a START to before first sentence
    for line in f:
        if line != '\n':
            line = line.strip('\n').split(' ')
            state = line[1]
            if state not in state_list:
                state_list.append(state)
    return state_list

# state_list = get_state_list('SG/train')
# state_list

In [6]:
em_param_table_UNK , unk_list = gen_emission_param_table_UNK('EN/train', 'EN/dev.in')
print(unk_list)
em_param_table_UNK

updating emission_count_table and y_count_dic
updating emission_param_table
unl_list is:  ["'71" '.50' '0.75' ... 'worthwhile' 'wraps' 'wrongly']
y_count_dic is:  {'B-NP': 47305, 'I-NP': 54591, 'B-VP': 18261, 'B-ADVP': 3565, 'B-ADJP': 1751, 'I-ADJP': 574, 'B-PP': 18387, 'O': 23872, 'B-SBAR': 1899, 'I-VP': 10159, 'I-ADVP': 363, 'B-PRT': 468, 'I-PP': 223, 'B-CONJP': 49, 'I-CONJP': 64, 'B-INTJ': 26, 'I-INTJ': 7, 'I-SBAR': 48, 'B-UCP': 1, 'I-UCP': 4, 'B-LST': 11}
["'71" '.50' '0.75' ... 'worthwhile' 'wraps' 'wrongly']


,Municipal,bonds,are,generally,a,bit,safer,than,corporate,in,...,Through,customs,gunslinging,agility,enriching,Suits,local-government,peering,essay,UNK
B-NP,0.000021,0.000444,0.000021,0.000042,0.075869,0.000000,0.000000,0.000000,0.000592,0.000021,...,0.000000,0.000021,0.000000,0.000021,0.000000,0.000021,0.000021,0.000000,0.000000,0.000011
I-NP,0.000018,0.001813,0.000000,0.000000,0.001612,0.000201,0.000000,0.001905,0.000385,0.000037,...,0.000000,0.000000,0.000018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000018,0.000009
B-VP,0.000000,0.000000,0.037073,0.000110,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000055,0.000000,0.000000,0.000055,0.000000,0.000027
B-ADVP,0.000000,0.000000,0.000000,0.003366,0.000000,0.000000,0.000000,0.000000,0.000000,0.004768,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000140
B-ADJP,0.000000,0.000000,0.000000,0.000000,0.001713,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000285
I-ADJP,0.000000,0.000000,0.000000,0.000000,0.000000,0.003481,0.001741,0.003481,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000870
B-PP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006961,0.000000,0.155649,...,0.000054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000027
O,0.000000,0.000000,0.000084,0.000000,0.000586,0.000000,0.000000,0.000126,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000021
B-SBAR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022111,0.000000,0.004738,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000263
I-VP,0.000000,0.000000,0.000098,0.000197,0.000000,0.000000,0.000000,0.000492,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000049


In [7]:
tr_param_table = gen_transition_param_table('EN/train')
tr_param_table

Generating transition parameter table


,B-NP,I-NP,B-VP,B-ADVP,B-ADJP,I-ADJP,B-PP,O,STOP,B-SBAR,...,B-PRT,I-PP,B-CONJP,I-CONJP,B-INTJ,I-INTJ,I-SBAR,B-UCP,I-UCP,B-LST
START,0.648049,0.000000,0.018661,0.054287,0.003262,0.000000,0.108704,0.141850,0.000000,0.022576,...,0.000000,0.000000,0.000261,0.000000,0.001305,0.000000,0.000000,0.000000,0.00,0.001044
B-NP,0.028898,0.684706,0.130303,0.009809,0.003213,0.000000,0.058007,0.080964,0.000233,0.003403,...,0.000359,0.000000,0.000085,0.000000,0.000000,0.000000,0.000000,0.000021,0.00,0.000000
I-NP,0.047645,0.406679,0.134912,0.015332,0.004103,0.000000,0.156509,0.227327,0.000788,0.006375,...,0.000128,0.000000,0.000201,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
B-VP,0.345217,0.000000,0.007229,0.031214,0.039209,0.000000,0.098735,0.067411,0.000055,0.025574,...,0.011171,0.000000,0.000164,0.000000,0.000110,0.000000,0.000000,0.000000,0.00,0.000000
B-ADVP,0.210379,0.000000,0.215989,0.016269,0.016550,0.000000,0.170547,0.265358,0.000842,0.016269,...,0.000281,0.000000,0.000561,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
B-ADJP,0.051970,0.000000,0.110794,0.015991,0.001142,0.279840,0.244432,0.256996,0.000571,0.037693,...,0.000571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
I-ADJP,0.088850,0.000000,0.069686,0.013937,0.012195,0.146341,0.285714,0.320557,0.001742,0.060976,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
B-PP,0.928047,0.000000,0.026595,0.003318,0.002611,0.000000,0.018491,0.008484,0.000218,0.000979,...,0.000000,0.011258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
O,0.347185,0.000000,0.115030,0.029197,0.008755,0.000000,0.050142,0.113522,0.318281,0.016128,...,0.000042,0.000000,0.001047,0.000000,0.000586,0.000000,0.000000,0.000000,0.00,0.000084
B-SBAR,0.872565,0.000000,0.038441,0.008952,0.003160,0.000000,0.014218,0.028436,0.000000,0.008425,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025276,0.000000,0.00,0.000527


In [94]:
def viterbi(em_param_table, tr_param_table, unk_list, sentence, state_list):
    # create the 2-D pi(j,u) table, initialize all values as 0 at beginning
    # sentence: a list storing all words in a sentence as its element
    # state_list : a list storing all the states except START and STOP
    start_node = 1
    end_node = np.array([0,0,0])
    
    # len(sentence)-1 becault first layer is actually a scalar
    s = (len(state_list),len(sentence)-1,3)
    
    # a 3d matrix with each row as the states, column as the word, 
    # each element is a 1x3 array to store top 3 positions 
    # pi_table.shape[0] -> rows (states)
    # pi_table.shape[1] -> columns (words)
    # pi_table.shape[2] -> elements (top 3 paths)
    pi_table = np.zeros(s)
    #print(pi_table)
    
    # Moving forward

    # from start to position 1 
    layer_1 = np.array([])
    # find values for the nodes at position 1 (first word)
    for i in range(len(state_list)):
        value = start_node * tr_param_table[state_list[i]]['START'] * get_emission_parameter_UNK(em_param_table,unk_list, sentence[0], state_list[i])
        layer_1 = np.append(layer_1,value)

    
    # find top 3 values for the nodes at position 2 (second word)
    # note the first column of pi_table corresponds to the second word since for the first word, it only has top 1 path
    
    for next_layer_index in range(len(state_list)):
        temp_result = np.array([])
        for curr_layer_index in range(len(state_list)):
            value = layer_1[curr_layer_index] * tr_param_table[state_list[next_layer_index]][state_list[curr_layer_index]] * get_emission_parameter_UNK(em_param_table,unk_list, sentence[1], state_list[next_layer_index])
            temp_result = np.append(temp_result,value)

            
        temp_result = temp_result[np.argsort(temp_result)[-3:]]
        temp_result = np.sort(temp_result)[::-1]
        pi_table[next_layer_index][0] = temp_result

    
    for i in range(pi_table.shape[1]-2):
        word_curr = sentence[i+1]
        word_next = sentence[i+2]
        
        for next_layer_index in range(len(state_list)):
            temp_result = np.array([])
            for curr_layer_index in range(len(state_list)):

                #for each 3 pi scores for each node
                for j in range(pi_table.shape[2]):
                    pi_value = pi_table[curr_layer_index][i][j]
                    value = pi_value * tr_param_table[state_list[next_layer_index]][state_list[curr_layer_index]] * get_emission_parameter_UNK(em_param_table,unk_list, word_next, state_list[next_layer_index])
                    temp_result = np.append(temp_result,value)

            temp_result = temp_result[np.argsort(temp_result)[-3:]]
            temp_result = np.sort(temp_result)[::-1]

            pi_table[next_layer_index][i+1] = temp_result

    
    
    # from n node to STOP
    temp_result = temp_result = np.array([])

    for curr_layer_index in range(len(state_list)):
        for j in range(pi_table.shape[2]):
            pi_value = pi_table[curr_layer_index][pi_table.shape[1]-1][j]
            value = pi_value * tr_param_table['STOP'][state_list[curr_layer_index]]
            temp_result = np.append(temp_result, value)
    
    temp_result = temp_result[np.argsort(temp_result)[-3:]]
    temp_result = np.sort(temp_result)[::-1]
    stop_node = temp_result.copy()
    
    
    optimum_path = []  # this is 3rd best path actually
    
    # Moving backward
    
    #the final value at STOP for the third optimal path
    path_3_value = np.min(stop_node)

    value_path = 0
    
    found_path = False
    for curr_layer_index in range(len(state_list)):
        if found_path:
            break
        for j in range(pi_table.shape[2]):
            pi_value = pi_table[curr_layer_index][pi_table.shape[1]-1][j]
            value = pi_value * tr_param_table['STOP'][state_list[curr_layer_index]]
            if value == path_3_value:
                value_path = pi_value
                node = state_list[curr_layer_index]
                optimum_path.insert(0,node)
                found_path = True
                break

    # moving back from n-1 to 3rd word
    for i in range(pi_table.shape[1]-2, -1, -1):
        word_next = sentence[i+2]
        for curr_layer_index in range(len(state_list)):
            path_found_temp = False
            for j in range(pi_table.shape[2]):
                pi_value = pi_table[curr_layer_index][i][j]
                value = pi_value * tr_param_table[optimum_path[0]][state_list[curr_layer_index]] * get_emission_parameter_UNK(em_param_table,unk_list, word_next, optimum_path[0])
                if value == value_path:
                    node = state_list[curr_layer_index]
                    optimum_path.insert(0,node)
                    value_path = pi_value
                    path_found_temp = True
                    break
            if path_found_temp:
                break
    
    #from 2nd word to 1st word
    
    for i in range(len(layer_1)):
        pi_value = layer_1[i]
        value = pi_value * tr_param_table[optimum_path[0]][state_list[i]] * get_emission_parameter_UNK(em_param_table,unk_list, sentence[1], optimum_path[0])
        if value == value_path:
            node = state_list[i]
            optimum_path.insert(0,node)
            
    return optimum_path # its called optimum_path but it is actually the 3rd best path
    

In [95]:
sentence_list = get_sentence_list('EN/dev.in')
sentence = sentence_list[0]
print(sentence)
state_list = get_state_list('EN/train')
print(state_list)
print(len(sentence))


['HBO', 'has', 'close', 'to', '24', 'million', 'subscribers', 'to', 'its', 'HBO', 'and', 'Cinemax', 'networks', ',', 'while', 'Showtime', 'and', 'its', 'sister', 'service', ',', 'The', 'Movie', 'Channel', ',', 'have', 'only', 'about', '10', 'million', ',', 'according', 'to', 'Paul', 'Kagan', 'Associates', ',', 'a', 'Carmel', ',', 'Calif.', ',', 'research', 'firm', '.']
['B-NP', 'I-NP', 'B-VP', 'B-ADVP', 'B-ADJP', 'I-ADJP', 'B-PP', 'O', 'B-SBAR', 'I-VP', 'I-ADVP', 'B-PRT', 'I-PP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-LST']
45


In [96]:
third_path = viterbi(em_param_table_UNK, tr_param_table, unk_list, sentence, state_list)
print(third_path)

['B-NP', 'B-VP', 'I-VP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'O', 'B-NP', 'I-NP', 'O', 'B-SBAR', 'B-NP', 'O', 'B-NP', 'I-NP', 'I-NP', 'O', 'B-NP', 'I-NP', 'I-NP', 'O', 'B-VP', 'I-VP', 'B-PP', 'B-NP', 'I-NP', 'O', 'B-PP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'O', 'B-NP', 'B-UCP', 'I-UCP', 'B-NP', 'O', 'B-NP', 'B-NP', 'B-NP']


In [92]:
def gen_state(train_path, input_path, output_path, em_param_table, tr_param_table, unk_list ):
    sentence_list = get_sentence_list(input_path)
    state_list = get_state_list(train_path)
    #viterbi(em_param_table, tr_param_table, unk_list, sentence, state_list):
    with open(output_path, 'w', encoding="utf8") as f:
        #for sentence in sentence_list:
        print("No. of sentence: ",len(sentence_list))
        for i in range(len(sentence_list)):
            sentence = sentence_list[i]
            print(i)
            #pi_table = create_pi_table(sentence, state_list)
            optimum_state = viterbi( em_param_table, tr_param_table, unk_list, sentence,state_list)
            for i in range (len(sentence)):
                #print("sentence: ", sentence)
                #print("optimum path: ", optimum_state)
                output = sentence[i] + ' ' + optimum_state[i]
                f.write(output)
                f.write('\n')
            f.write('\n')

In [93]:
gen_state('EN/train','EN/dev.in', 'EN/dev.p4.out', em_param_table_UNK, tr_param_table, unk_list )

No. of sentence:  1094
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


KeyboardInterrupt: 